In [ ]:
import os
import numpy as np
import tkinter as tk
from tkinter import filedialog
from PIL import Image, ImageTk
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K

# Set the path to the main directory containing subdirectories with images
main_dir = 'C:/Users/Plaksha/Desktop/fma_img/fma_img'

# Define image dimensions and batch size
img_height = 100
img_width = 100
batch_size = 32

# Function to load images and split them into train, validation, and test sets
def load_data(main_dir, train_size=20, val_size=3, test_size=2):
    train_images = []
    val_images = []
    test_images = []
    labels = []

    for i, subdir in enumerate(os.listdir(main_dir)):
        subdir_path = os.path.join(main_dir, subdir)
        images = os.listdir(subdir_path)
        np.random.shuffle(images)

        # Split images into train, validation, and test sets
        train_images.extend([os.path.join(subdir_path, img) for img in images[:train_size]])
        val_images.extend([os.path.join(subdir_path, img) for img in images[train_size:train_size+val_size]])
        test_images.extend([os.path.join(subdir_path, img) for img in images[train_size+val_size:train_size+val_size+test_size]])
        labels.extend([i] * (train_size + val_size + test_size))

    return train_images, val_images, test_images, labels

train_images, val_images, test_images, labels = load_data(main_dir)

# Load the model
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(len(os.listdir(main_dir)), activation=None)  # No activation for logits
])

# Define Angular Margin Loss function
class AngularMarginLoss(CategoricalCrossentropy):
    def __init__(self, alpha=0.2, scale=30.0, margin=0.5, **kwargs):
        super(AngularMarginLoss, self).__init__(**kwargs)
        self.alpha = alpha
        self.scale = scale
        self.margin = margin

    def __call__(self, y_true, y_pred):
        # Extract the logits of the ground truth classes
        logits = tf.reduce_sum(y_true * y_pred, axis=-1)

        # Calculate cos(theta) and theta
        cos_theta = logits / (tf.norm(y_pred, axis=-1) * tf.norm(y_true, axis=-1) + 1e-10)
        theta = tf.acos(tf.clip_by_value(cos_theta, -1.0 + K.epsilon(), 1.0 - K.epsilon()))

        # Calculate target cos and sin values
        target_cos = tf.cos(tf.acos(cos_theta) + self.margin)
        target_sin = tf.sin(tf.acos(cos_theta) + self.margin)

        # Calculate final logits
        final_logits = self.scale * ((cos_theta * target_cos) - (theta * target_sin))

        # Apply softmax
        return super(AngularMarginLoss, self).__call__(y_true, final_logits)

# Compile the model with Angular Margin Loss
model.compile(optimizer=Adam(),
              loss=AngularMarginLoss(),
              metrics=['accuracy'])

# Function to preprocess the uploaded image
def preprocess_image(image_path):
    img = load_img(image_path, target_size=(img_height, img_width))
    img_array = img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    return img_array

# Function to calculate embeddings
def calculate_embeddings(image_path):
    img_array = preprocess_image(image_path)
    embeddings = model.predict(img_array)
    return embeddings

# Function to open file dialog and get the image path
def browse_file():
    file_path = filedialog.askopenfilename()
    image_path_var.set(file_path)
    if file_path:
        img = Image.open(file_path)
        img = img.resize((200, 200), Image.ANTIALIAS)
        img = ImageTk.PhotoImage(img)
        image_label.config(image=img)
        image_label.image = img

# Function to match embeddings and determine class
def match_embeddings():
    image_path = image_path_var.get()
    if image_path:
        uploaded_embeddings = calculate_embeddings(image_path)
        min_distance = float('inf')
        min_class = None
        for subdir in os.listdir(main_dir):
            subdir_path = os.path.join(main_dir, subdir)
            for img_file in os.listdir(subdir_path):
                img_path = os.path.join(subdir_path, img_file)
                embeddings = calculate_embeddings(img_path)
                distance = np.linalg.norm(uploaded_embeddings - embeddings)
                if distance < min_distance:
                    min_distance = distance
                    min_class = subdir
        result_var.set(f"The song clip belongs to class: {min_class}")

# Create tkinter window
root = tk.Tk()
root.title("Image Classification")

# Create widgets
browse_button = tk.Button(root, text="Upload Image", command=browse_file)
browse_button.pack(pady=10)

image_label = tk.Label(root)
image_label.pack(pady=10)

image_path_var = tk.StringVar()
image_path_entry = tk.Entry(root, textvariable=image_path_var, state='readonly')
image_path_entry.pack(pady=10)

classify_button = tk.Button(root, text="Classify Image", command=match_embeddings)
classify_button.pack(pady=10)

result_var = tk.StringVar()
result_label = tk.Label(root, textvariable=result_var)
result_label.pack(pady=10)

# Function to close the tkinter window
def close_window():
    root.destroy()

# Bind closing event to close_window function
root.protocol("WM_DELETE_WINDOW", close_window)

# Run the tkinter event loop
root.mainloop()
